In [1]:
import matplotlib.pyplot as plt
import sys
import numpy as np
import pandas as pd 
import netCDF4 as nc
from cartopy import config
import cartopy.crs as ccrs
import os
import shutil
import tempfile

In [2]:
np.set_printoptions(threshold=sys.maxsize)
os.chdir(r"C:\Users\eliza\Documents\Uni\Project\Data")

minlat = 90  # Set Latitude bounds (90 - 180 is NH, 0 - 90 is SH)
maxlat = 180  
minlong = 0  # Set Longitude Bounds
maxlong = 360  # 0-360 for checking latitudes

## 1984

In [3]:
#1984
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))  # Stores DJF for eaach year
WSX_all = np.zeros((373,181,360))

In [4]:
fn = r"WS\1984.nc"  # Select WS data for each year
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360) # Set the axes
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]  # Identify correct dataset for each month
jan6hr = jan[0::2]  # Make data 6 hourly
jan6hr = np.ma.filled(jan6hr, fill_value=0)  # Fill in masked data
jantranslated = np.roll(jan6hr, 180, axis=2)  # Translate WS data by 180 degrees
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [5]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1984\JAN"):  # Run through the fronts files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1984\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # Empty array for storing WS data
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))  # Shape must match cdi

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:  # Selects gridboxes containing matched fronts
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:  # Selects unmatched fronts
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX


In [6]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1984\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1984\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [7]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1984\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1984\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [8]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [9]:
WS1984 = np.zeros(11)  # Count occurences of each WS for with DI
WS1984[0] = ws.count(1)
WS1984[1] = ws.count(2)
WS1984[2] = ws.count(3)
WS1984[3] = ws.count(4)
WS1984[4] = ws.count(5)
WS1984[5] = ws.count(6)
WS1984[6] = ws.count(7)
WS1984[7] = ws.count(8)
WS1984[8] = ws.count(9)
WS1984[9] = ws.count(10)
WS1984[10] = ws.count(11)

WS1984x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1984x[0] = wsx.count(1)
WS1984x[1] = wsx.count(2)
WS1984x[2] = wsx.count(3)
WS1984x[3] = wsx.count(4)
WS1984x[4] = wsx.count(5)
WS1984x[5] = wsx.count(6)
WS1984x[6] = wsx.count(7)
WS1984x[7] = wsx.count(8)
WS1984x[8] = wsx.count(9)
WS1984x[9] = wsx.count(10)
WS1984x[10] = wsx.count(11)

print(WS1984, WS1984x)

[ 3555.  9974.  2221.   435.  6223.   509. 12495.  4050.  3369.  1257.
   497.] [ 3853.  9493.  2631.  2064.  5943.   633. 14849.  3899.  2203.  1240.
   529.]


## 1985

In [10]:
#1985
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [11]:
fn = r"WS\1985.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [12]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1985\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1985\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [13]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1985\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1985\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [14]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1985\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1985\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [15]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [16]:
WS1985 = np.zeros(11)  # Count occurences of each WS for with DI
WS1985[0] = ws.count(1)
WS1985[1] = ws.count(2)
WS1985[2] = ws.count(3)
WS1985[3] = ws.count(4)
WS1985[4] = ws.count(5)
WS1985[5] = ws.count(6)
WS1985[6] = ws.count(7)
WS1985[7] = ws.count(8)
WS1985[8] = ws.count(9)
WS1985[9] = ws.count(10)
WS1985[10] = ws.count(11)

WS1985x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1985x[0] = wsx.count(1)
WS1985x[1] = wsx.count(2)
WS1985x[2] = wsx.count(3)
WS1985x[3] = wsx.count(4)
WS1985x[4] = wsx.count(5)
WS1985x[5] = wsx.count(6)
WS1985x[6] = wsx.count(7)
WS1985x[7] = wsx.count(8)
WS1985x[8] = wsx.count(9)
WS1985x[9] = wsx.count(10)
WS1985x[10] = wsx.count(11)

print(WS1985, WS1985x)

[ 3895.  8969.  3306.   271.  4094.   591. 12254.  4677.  2678.  1107.
   432.] [ 3239.  7690.  4179.  1702.  3425.  1012. 12434.  4403.  1496.   545.
   448.]


## 1986

In [17]:
#1986
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [18]:
fn = r"WS\1986.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [19]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1986\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1986\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [20]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1986\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1986\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [21]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1986\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1986\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [22]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [23]:
WS1986 = np.zeros(11)  # Count occurences of each WS for with DI
WS1986[0] = ws.count(1)
WS1986[1] = ws.count(2)
WS1986[2] = ws.count(3)
WS1986[3] = ws.count(4)
WS1986[4] = ws.count(5)
WS1986[5] = ws.count(6)
WS1986[6] = ws.count(7)
WS1986[7] = ws.count(8)
WS1986[8] = ws.count(9)
WS1986[9] = ws.count(10)
WS1986[10] = ws.count(11)

WS1986x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1986x[0] = wsx.count(1)
WS1986x[1] = wsx.count(2)
WS1986x[2] = wsx.count(3)
WS1986x[3] = wsx.count(4)
WS1986x[4] = wsx.count(5)
WS1986x[5] = wsx.count(6)
WS1986x[6] = wsx.count(7)
WS1986x[7] = wsx.count(8)
WS1986x[8] = wsx.count(9)
WS1986x[9] = wsx.count(10)
WS1986x[10] = wsx.count(11)

print(WS1986, WS1986x)

[ 5460. 10653.  3864.   396.  4593.   766. 14699.  7432.  3208.  1377.
   550.] [ 4863.  9525.  4474.  2134.  4476.  1241. 15440.  5105.  1550.   730.
   700.]


## 1987

In [24]:
#1987
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [25]:
fn = r"WS\1987.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [26]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1987\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1987\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [27]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1987\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1987\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [28]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1987\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1987\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [29]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [30]:
WS1987 = np.zeros(11)  # Count occurences of each WS for with DI
WS1987[0] = ws.count(1)
WS1987[1] = ws.count(2)
WS1987[2] = ws.count(3)
WS1987[3] = ws.count(4)
WS1987[4] = ws.count(5)
WS1987[5] = ws.count(6)
WS1987[6] = ws.count(7)
WS1987[7] = ws.count(8)
WS1987[8] = ws.count(9)
WS1987[9] = ws.count(10)
WS1987[10] = ws.count(11)

WS1987x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1987x[0] = wsx.count(1)
WS1987x[1] = wsx.count(2)
WS1987x[2] = wsx.count(3)
WS1987x[3] = wsx.count(4)
WS1987x[4] = wsx.count(5)
WS1987x[5] = wsx.count(6)
WS1987x[6] = wsx.count(7)
WS1987x[7] = wsx.count(8)
WS1987x[8] = wsx.count(9)
WS1987x[9] = wsx.count(10)
WS1987x[10] = wsx.count(11)

print(WS1987, WS1987x)

[ 5035.  9712.  3561.   399.  5098.   795. 15144.  7956.  3235.  1433.
   792.] [ 4565.  9745.  5404.  2412.  3966.  1511. 16121.  5474.  1333.   662.
   667.]


## 1988

In [31]:
#1988
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [32]:
fn = r"WS\1988.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [33]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1988\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1988\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [34]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1988\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1988\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [35]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1988\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1988\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [36]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [37]:
WS1988 = np.zeros(11)  # Count occurences of each WS for with DI
WS1988[0] = ws.count(1)
WS1988[1] = ws.count(2)
WS1988[2] = ws.count(3)
WS1988[3] = ws.count(4)
WS1988[4] = ws.count(5)
WS1988[5] = ws.count(6)
WS1988[6] = ws.count(7)
WS1988[7] = ws.count(8)
WS1988[8] = ws.count(9)
WS1988[9] = ws.count(10)
WS1988[10] = ws.count(11)

WS1988x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1988x[0] = wsx.count(1)
WS1988x[1] = wsx.count(2)
WS1988x[2] = wsx.count(3)
WS1988x[3] = wsx.count(4)
WS1988x[4] = wsx.count(5)
WS1988x[5] = wsx.count(6)
WS1988x[6] = wsx.count(7)
WS1988x[7] = wsx.count(8)
WS1988x[8] = wsx.count(9)
WS1988x[9] = wsx.count(10)
WS1988x[10] = wsx.count(11)

print(WS1988, WS1988x)

[ 3800.  8975.  3358.   417.  3617.   664. 12176.  6426.  2445.  1258.
   584.] [ 4091.  8442.  3939.  1802.  3323.  1216. 12087.  4759.  1260.   718.
   539.]


## 1989

In [38]:
#1989
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [39]:
fn = r"WS\1989.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [40]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1989\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1989\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [41]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1989\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1989\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [42]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1989\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1989\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [43]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [44]:
WS1989 = np.zeros(11)  # Count occurences of each WS for with DI
WS1989[0] = ws.count(1)
WS1989[1] = ws.count(2)
WS1989[2] = ws.count(3)
WS1989[3] = ws.count(4)
WS1989[4] = ws.count(5)
WS1989[5] = ws.count(6)
WS1989[6] = ws.count(7)
WS1989[7] = ws.count(8)
WS1989[8] = ws.count(9)
WS1989[9] = ws.count(10)
WS1989[10] = ws.count(11)

WS1989x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1989x[0] = wsx.count(1)
WS1989x[1] = wsx.count(2)
WS1989x[2] = wsx.count(3)
WS1989x[3] = wsx.count(4)
WS1989x[4] = wsx.count(5)
WS1989x[5] = wsx.count(6)
WS1989x[6] = wsx.count(7)
WS1989x[7] = wsx.count(8)
WS1989x[8] = wsx.count(9)
WS1989x[9] = wsx.count(10)
WS1989x[10] = wsx.count(11)

print(WS1989, WS1989x)

[ 4023.  9816.  2471.   648.  4276.   598. 12638.  5758.  2812.  1463.
   626.] [ 3204.  8778.  3108.  2779.  4057.   897. 14839.  5496.  1737.  1054.
   908.]


## 1990

In [45]:
#1990
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [46]:
fn = r"WS\1990.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [47]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1990\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1990\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [48]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1990\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1990\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [49]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1990\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1990\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [50]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [51]:
WS1990 = np.zeros(11)  # Count occurences of each WS for with DI
WS1990[0] = ws.count(1)
WS1990[1] = ws.count(2)
WS1990[2] = ws.count(3)
WS1990[3] = ws.count(4)
WS1990[4] = ws.count(5)
WS1990[5] = ws.count(6)
WS1990[6] = ws.count(7)
WS1990[7] = ws.count(8)
WS1990[8] = ws.count(9)
WS1990[9] = ws.count(10)
WS1990[10] = ws.count(11)

WS1990x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1990x[0] = wsx.count(1)
WS1990x[1] = wsx.count(2)
WS1990x[2] = wsx.count(3)
WS1990x[3] = wsx.count(4)
WS1990x[4] = wsx.count(5)
WS1990x[5] = wsx.count(6)
WS1990x[6] = wsx.count(7)
WS1990x[7] = wsx.count(8)
WS1990x[8] = wsx.count(9)
WS1990x[9] = wsx.count(10)
WS1990x[10] = wsx.count(11)

print(WS1990, WS1990x)

[ 4162. 10692.  3456.   654.  5495.  1001. 14054.  7048.  3797.  1443.
   585.] [ 3484. 10392.  3184.  3264.  4632.   972. 13258.  5305.  1828.   902.
   515.]


## 1991

In [52]:
#1991
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [53]:
fn = r"WS\1991.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [54]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1991\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1991\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [55]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1991\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1991\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [56]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1991\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1991\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [57]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [58]:
WS1991 = np.zeros(11)  # Count occurences of each WS for with DI
WS1991[0] = ws.count(1)
WS1991[1] = ws.count(2)
WS1991[2] = ws.count(3)
WS1991[3] = ws.count(4)
WS1991[4] = ws.count(5)
WS1991[5] = ws.count(6)
WS1991[6] = ws.count(7)
WS1991[7] = ws.count(8)
WS1991[8] = ws.count(9)
WS1991[9] = ws.count(10)
WS1991[10] = ws.count(11)

WS1991x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1991x[0] = wsx.count(1)
WS1991x[1] = wsx.count(2)
WS1991x[2] = wsx.count(3)
WS1991x[3] = wsx.count(4)
WS1991x[4] = wsx.count(5)
WS1991x[5] = wsx.count(6)
WS1991x[6] = wsx.count(7)
WS1991x[7] = wsx.count(8)
WS1991x[8] = wsx.count(9)
WS1991x[9] = wsx.count(10)
WS1991x[10] = wsx.count(11)

print(WS1991, WS1991x)

[ 4446.  9221.  1561.   489.  4582.   374. 12139.  6540.  3081.  1094.
   607.] [ 4808. 11248.  3435.  3119.  4872.   870. 16898.  6697.  2290.   968.
   802.]


## 1992

In [59]:
#1992
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [60]:
fn = r"WS\1992.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [61]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1992\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1992\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [62]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1992\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1992\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [63]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1992\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1992\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [64]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [65]:
WS1992 = np.zeros(11)  # Count occurences of each WS for with DI
WS1992[0] = ws.count(1)
WS1992[1] = ws.count(2)
WS1992[2] = ws.count(3)
WS1992[3] = ws.count(4)
WS1992[4] = ws.count(5)
WS1992[5] = ws.count(6)
WS1992[6] = ws.count(7)
WS1992[7] = ws.count(8)
WS1992[8] = ws.count(9)
WS1992[9] = ws.count(10)
WS1992[10] = ws.count(11)

WS1992x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1992x[0] = wsx.count(1)
WS1992x[1] = wsx.count(2)
WS1992x[2] = wsx.count(3)
WS1992x[3] = wsx.count(4)
WS1992x[4] = wsx.count(5)
WS1992x[5] = wsx.count(6)
WS1992x[6] = wsx.count(7)
WS1992x[7] = wsx.count(8)
WS1992x[8] = wsx.count(9)
WS1992x[9] = wsx.count(10)
WS1992x[10] = wsx.count(11)

print(WS1992, WS1992x)

[ 4865. 10469.  1975.   337.  4264.   419. 14206. 10404.  3784.  1366.
   607.] [ 4275.  9822.  2869.  2037.  4401.   883. 16084.  8365.  2127.   980.
   958.]


## 1993

In [66]:
#1993
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [67]:
fn = r"WS\1993.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [68]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1993\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1993\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [69]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1993\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1993\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [70]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1993\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1993\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [71]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [72]:
WS1993 = np.zeros(11)  # Count occurences of each WS for with DI
WS1993[0] = ws.count(1)
WS1993[1] = ws.count(2)
WS1993[2] = ws.count(3)
WS1993[3] = ws.count(4)
WS1993[4] = ws.count(5)
WS1993[5] = ws.count(6)
WS1993[6] = ws.count(7)
WS1993[7] = ws.count(8)
WS1993[8] = ws.count(9)
WS1993[9] = ws.count(10)
WS1993[10] = ws.count(11)

WS1993x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1993x[0] = wsx.count(1)
WS1993x[1] = wsx.count(2)
WS1993x[2] = wsx.count(3)
WS1993x[3] = wsx.count(4)
WS1993x[4] = wsx.count(5)
WS1993x[5] = wsx.count(6)
WS1993x[6] = wsx.count(7)
WS1993x[7] = wsx.count(8)
WS1993x[8] = wsx.count(9)
WS1993x[9] = wsx.count(10)
WS1993x[10] = wsx.count(11)

print(WS1993, WS1993x)

[ 4575.  8501.  3215.   317.  3609.   952. 11768.  5821.  2851.  1165.
   536.] [ 5104. 11680.  4762.  1967.  4287.  1443. 16362.  6160.  2078.   978.
   842.]


## 1994 Undefined for DJF

In [73]:
# 1994
# a = -1  # counter for jan
# b = 123  # counter for feb
# c = 247  # counter for dec

# WS_all = np.zeros((373,181,360))
# WSX_all = np.zeros((373,181,360))

In [74]:
# fn = r"WS\1994.nc"
# ds = nc.Dataset(fn)

# lon = np.linspace(-180, 180, 360)
# lat = np.linspace(-90, 90, 180)
# ws = ds.variables['ws'][:]

# jan = ds.variables['january'][:]
# jan6hr = jan[0::2]
# jan6hr = np.ma.filled(jan6hr, fill_value=0)
# jantranslated = np.roll(jan6hr, 180, axis=2)
# jansize = np.size(jan6hr, 0)

# feb = ds.variables['february'][:]
# feb6hr = feb[0::2]
# feb6hr = np.ma.filled(feb6hr, fill_value=0)
# febtranslated = np.roll(feb6hr, 180, axis=2)
# febsize = np.size(feb6hr, 0)

# dec = ds.variables['december'][:]
# dec6hr = dec[0::2]
# dec6hr = np.ma.filled(dec6hr, fill_value=0)
# dectranslated = np.roll(dec6hr, 180, axis=2)
# decsize = np.size(dec6hr, 0)

In [75]:
# for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1994\JAN"):  # Run through the files in each folder
#     a += 1
#     if filename.endswith(".nc"):            
#         fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1994\JAN", filename)
#         ds1 = nc.Dataset(fn1)

#         lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
#         lon1 = ds1.variables['longitude'][:]
#         ct = ds1.variables['coldtrail'][:]
#         dic = ds1.variables['di_coldtrail'][:]
#         cdi = ds1.variables['coldtrail_di'][:]
        
#         WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
#         WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

#         for i in range(minlat, maxlat):
#             for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
#                 if cdi[i,j] != 0:
#                     WS[i,j] = jantranslated[a,j,i]                    
#                 elif ct[i,j] != 0:
#                     WSX[i,j] = jantranslated[a,j,i]
                    
#         WS_all[a,:,:] = WS
#         WSX_all[a,:,:] = WSX

In [76]:
# a = -1
# for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1994\FEB"):  # Run through the files in each folder
#     a += 1
#     b += 1
#     if filename.endswith(".nc"):            
#         fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1994\FEB", filename)
#         ds1 = nc.Dataset(fn1)

#         lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
#         lon1 = ds1.variables['longitude'][:]
#         ct = ds1.variables['coldtrail'][:]
#         dic = ds1.variables['di_coldtrail'][:]
#         cdi = ds1.variables['coldtrail_di'][:]
        
#         WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
#         WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

#         for i in range(minlat, maxlat):
#             for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
#                 if cdi[i,j] != 0:
#                     WS[i,j] = febtranslated[a,j,i]                    
#                 elif ct[i,j] != 0:
#                     WSX[i,j] = febtranslated[a,j,i]

#         WS_all[b,:,:] = WS
#         WSX_all[b,:,:] = WSX

In [77]:
# a = -1
# for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1994\DEC"):  # Run through the files in each folder
#     a += 1
#     c += 1
#     if filename.endswith(".nc"):            
#         fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1994\DEC", filename)
#         ds1 = nc.Dataset(fn1)

#         lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
#         lon1 = ds1.variables['longitude'][:]
#         ct = ds1.variables['coldtrail'][:]
#         dic = ds1.variables['di_coldtrail'][:]
#         cdi = ds1.variables['coldtrail_di'][:]
        
#         WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
#         WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

#         for i in range(minlat, maxlat):
#             for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
#                 if cdi[i,j] != 0:
#                     WS[i,j] = dectranslated[a,j,i]                    
#                 elif ct[i,j] != 0:
#                     WSX[i,j] = dectranslated[a,j,i]
                    
#         WS_all[c,:,:] = WS
#         WSX_all[c,:,:] = WSX

In [78]:
# ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
# ws = ws.tolist()
# wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
# wsx = wsx.tolist()

In [79]:
# WS1994 = np.zeros(11)  # Count occurences of each WS for with DI
# WS1994[0] = ws.count(1)
# WS1994[1] = ws.count(2)
# WS1994[2] = ws.count(3)
# WS1994[3] = ws.count(4)
# WS1994[4] = ws.count(5)
# WS1994[5] = ws.count(6)
# WS1994[6] = ws.count(7)
# WS1994[7] = ws.count(8)
# WS1994[8] = ws.count(9)
# WS1994[9] = ws.count(10)
# WS1994[10] = ws.count(11)

# WS1994x = np.zeros(11)  # Count occurences of each WS for w/o DI
# WS1994x[0] = wsx.count(1)
# WS1994x[1] = wsx.count(2)
# WS1994x[2] = wsx.count(3)
# WS1994x[3] = wsx.count(4)
# WS1994x[4] = wsx.count(5)
# WS1994x[5] = wsx.count(6)
# WS1994x[6] = wsx.count(7)
# WS1994x[7] = wsx.count(8)
# WS1994x[8] = wsx.count(9)
# WS1994x[9] = wsx.count(10)
# WS1994x[10] = wsx.count(11)

# print(WS1994, WS1994x)

## 1995

In [80]:
#1995
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [81]:
fn = r"WS\1995.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [82]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1995\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1995\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [83]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1995\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1995\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [84]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1995\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1995\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [85]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [86]:
WS1995 = np.zeros(11)  # Count occurences of each WS for with DI
WS1995[0] = ws.count(1)
WS1995[1] = ws.count(2)
WS1995[2] = ws.count(3)
WS1995[3] = ws.count(4)
WS1995[4] = ws.count(5)
WS1995[5] = ws.count(6)
WS1995[6] = ws.count(7)
WS1995[7] = ws.count(8)
WS1995[8] = ws.count(9)
WS1995[9] = ws.count(10)
WS1995[10] = ws.count(11)

WS1995x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1995x[0] = wsx.count(1)
WS1995x[1] = wsx.count(2)
WS1995x[2] = wsx.count(3)
WS1995x[3] = wsx.count(4)
WS1995x[4] = wsx.count(5)
WS1995x[5] = wsx.count(6)
WS1995x[6] = wsx.count(7)
WS1995x[7] = wsx.count(8)
WS1995x[8] = wsx.count(9)
WS1995x[9] = wsx.count(10)
WS1995x[10] = wsx.count(11)

print(WS1995, WS1995x)

[ 3269.  6975.  2726.   217.  3326.   697. 12450.  5087.  2664.   913.
   738.] [ 4324.  9395.  4398.  1276.  3937.  1093. 14462.  4147.  1945.   828.
   838.]


## 1996

In [87]:
#1996
a = -1  # counter for jan
b = 123  # counter for feb
c = 247  # counter for dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [88]:
fn = r"WS\1996.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [89]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1996\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1996\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [90]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1996\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1996\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [91]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1996\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1996\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [92]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [93]:
WS1996 = np.zeros(11)  # Count occurences of each WS for with DI
WS1996[0] = ws.count(1)
WS1996[1] = ws.count(2)
WS1996[2] = ws.count(3)
WS1996[3] = ws.count(4)
WS1996[4] = ws.count(5)
WS1996[5] = ws.count(6)
WS1996[6] = ws.count(7)
WS1996[7] = ws.count(8)
WS1996[8] = ws.count(9)
WS1996[9] = ws.count(10)
WS1996[10] = ws.count(11)

WS1996x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1996x[0] = wsx.count(1)
WS1996x[1] = wsx.count(2)
WS1996x[2] = wsx.count(3)
WS1996x[3] = wsx.count(4)
WS1996x[4] = wsx.count(5)
WS1996x[5] = wsx.count(6)
WS1996x[6] = wsx.count(7)
WS1996x[7] = wsx.count(8)
WS1996x[8] = wsx.count(9)
WS1996x[9] = wsx.count(10)
WS1996x[10] = wsx.count(11)

print(WS1996, WS1996x)

[ 2363. 10770.  2452.   338.  5026.   930. 14730.  7397.  3307.  2214.
   831.] [ 2302.  9178.  3910.  2336.  4992.  1373. 18434.  5951.  2195.  1417.
   978.]


## 1997

In [94]:
#1997
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [95]:
fn = r"WS\1997.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [96]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1997\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1997\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [97]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1997\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1997\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [98]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1997\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1997\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [99]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [100]:
WS1997 = np.zeros(11)  # Count occurences of each WS for with DI
WS1997[0] = ws.count(1)
WS1997[1] = ws.count(2)
WS1997[2] = ws.count(3)
WS1997[3] = ws.count(4)
WS1997[4] = ws.count(5)
WS1997[5] = ws.count(6)
WS1997[6] = ws.count(7)
WS1997[7] = ws.count(8)
WS1997[8] = ws.count(9)
WS1997[9] = ws.count(10)
WS1997[10] = ws.count(11)

WS1997x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1997x[0] = wsx.count(1)
WS1997x[1] = wsx.count(2)
WS1997x[2] = wsx.count(3)
WS1997x[3] = wsx.count(4)
WS1997x[4] = wsx.count(5)
WS1997x[5] = wsx.count(6)
WS1997x[6] = wsx.count(7)
WS1997x[7] = wsx.count(8)
WS1997x[8] = wsx.count(9)
WS1997x[9] = wsx.count(10)
WS1997x[10] = wsx.count(11)

print(WS1997, WS1997x)

[ 4035.  9961.  3243.   389.  4628.   940. 13956.  5560.  4042.  1657.
   611.] [ 3990.  9464.  4084.  2961.  4356.  1426. 15825.  4784.  2101.  1128.
  1035.]


## 1998

In [101]:
#1998
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [102]:
fn = r"WS\1998.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [103]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1998\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1998\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [104]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1998\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1998\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [105]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1998\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1998\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [106]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [107]:
WS1998 = np.zeros(11)  # Count occurences of each WS for with DI
WS1998[0] = ws.count(1)
WS1998[1] = ws.count(2)
WS1998[2] = ws.count(3)
WS1998[3] = ws.count(4)
WS1998[4] = ws.count(5)
WS1998[5] = ws.count(6)
WS1998[6] = ws.count(7)
WS1998[7] = ws.count(8)
WS1998[8] = ws.count(9)
WS1998[9] = ws.count(10)
WS1998[10] = ws.count(11)

WS1998x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1998x[0] = wsx.count(1)
WS1998x[1] = wsx.count(2)
WS1998x[2] = wsx.count(3)
WS1998x[3] = wsx.count(4)
WS1998x[4] = wsx.count(5)
WS1998x[5] = wsx.count(6)
WS1998x[6] = wsx.count(7)
WS1998x[7] = wsx.count(8)
WS1998x[8] = wsx.count(9)
WS1998x[9] = wsx.count(10)
WS1998x[10] = wsx.count(11)

print(WS1998, WS1998x)

[ 3084.  9489.  2594.   458.  5006.   634. 15175.  4910.  4171.  1612.
   822.] [ 3218. 12084.  3927.  1964.  5291.  1140. 17376.  4611.  2625.  1340.
   972.]


## 1999

In [108]:
#1999
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [109]:
fn = r"WS\1999.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [110]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1999\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1999\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [111]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1999\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1999\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [112]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_1999\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_1999\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [113]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [114]:
WS1999 = np.zeros(11)  # Count occurences of each WS for with DI
WS1999[0] = ws.count(1)
WS1999[1] = ws.count(2)
WS1999[2] = ws.count(3)
WS1999[3] = ws.count(4)
WS1999[4] = ws.count(5)
WS1999[5] = ws.count(6)
WS1999[6] = ws.count(7)
WS1999[7] = ws.count(8)
WS1999[8] = ws.count(9)
WS1999[9] = ws.count(10)
WS1999[10] = ws.count(11)

WS1999x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS1999x[0] = wsx.count(1)
WS1999x[1] = wsx.count(2)
WS1999x[2] = wsx.count(3)
WS1999x[3] = wsx.count(4)
WS1999x[4] = wsx.count(5)
WS1999x[5] = wsx.count(6)
WS1999x[6] = wsx.count(7)
WS1999x[7] = wsx.count(8)
WS1999x[8] = wsx.count(9)
WS1999x[9] = wsx.count(10)
WS1999x[10] = wsx.count(11)

print(WS1999, WS1999x)

[ 4351. 10409.  3743.   354.  4830.  1021. 14014.  6565.  3714.  1869.
   818.] [ 2515. 10114.  4694.  2615.  4524.  1570. 15657.  5099.  1836.  1038.
  1125.]


## 2000

In [115]:
#2000
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [116]:
fn = r"WS\2000.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [117]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2000\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2000\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [118]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2000\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2000\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [119]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2000\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2000\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [120]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [121]:
WS2000 = np.zeros(11)  # Count occurences of each WS for with DI
WS2000[0] = ws.count(1)
WS2000[1] = ws.count(2)
WS2000[2] = ws.count(3)
WS2000[3] = ws.count(4)
WS2000[4] = ws.count(5)
WS2000[5] = ws.count(6)
WS2000[6] = ws.count(7)
WS2000[7] = ws.count(8)
WS2000[8] = ws.count(9)
WS2000[9] = ws.count(10)
WS2000[10] = ws.count(11)

WS2000x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS2000x[0] = wsx.count(1)
WS2000x[1] = wsx.count(2)
WS2000x[2] = wsx.count(3)
WS2000x[3] = wsx.count(4)
WS2000x[4] = wsx.count(5)
WS2000x[5] = wsx.count(6)
WS2000x[6] = wsx.count(7)
WS2000x[7] = wsx.count(8)
WS2000x[8] = wsx.count(9)
WS2000x[9] = wsx.count(10)
WS2000x[10] = wsx.count(11)

print(WS2000, WS2000x)

[ 4406. 10491.  4633.   283.  4964.  1110. 15232.  5576.  3628.  1674.
   986.] [ 2742.  9284.  5091.  2248.  4740.  1324. 15938.  4119.  2281.  1158.
  1139.]


## 2001

In [122]:
#2001
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [123]:
fn = r"WS\2001.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [124]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2001\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2001\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX


In [125]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2001\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2001\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [126]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2001\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2001\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [127]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [128]:
WS2001 = np.zeros(11)  # Count occurences of each WS for with DI
WS2001[0] = ws.count(1)
WS2001[1] = ws.count(2)
WS2001[2] = ws.count(3)
WS2001[3] = ws.count(4)
WS2001[4] = ws.count(5)
WS2001[5] = ws.count(6)
WS2001[6] = ws.count(7)
WS2001[7] = ws.count(8)
WS2001[8] = ws.count(9)
WS2001[9] = ws.count(10)
WS2001[10] = ws.count(11)

WS2001x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS2001x[0] = wsx.count(1)
WS2001x[1] = wsx.count(2)
WS2001x[2] = wsx.count(3)
WS2001x[3] = wsx.count(4)
WS2001x[4] = wsx.count(5)
WS2001x[5] = wsx.count(6)
WS2001x[6] = wsx.count(7)
WS2001x[7] = wsx.count(8)
WS2001x[8] = wsx.count(9)
WS2001x[9] = wsx.count(10)
WS2001x[10] = wsx.count(11)

print(WS2001, WS2001x)

[ 5692. 12255.  3430.   352.  5536.   792. 15772.  6334.  4353.  2229.
   851.] [ 3383.  9663.  4050.  2258.  4789.  1116. 16885.  4148.  2029.  1076.
   794.]


## 2002

In [129]:
#2003
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [130]:
fn = r"WS\2000.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [131]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2002\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2002\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX


In [132]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2002\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2002\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [133]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2002\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2002\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [134]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [135]:
WS2002 = np.zeros(11)  # Count occurences of each WS for with DI
WS2002[0] = ws.count(1)
WS2002[1] = ws.count(2)
WS2002[2] = ws.count(3)
WS2002[3] = ws.count(4)
WS2002[4] = ws.count(5)
WS2002[5] = ws.count(6)
WS2002[6] = ws.count(7)
WS2002[7] = ws.count(8)
WS2002[8] = ws.count(9)
WS2002[9] = ws.count(10)
WS2002[10] = ws.count(11)

WS2002x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS2002x[0] = wsx.count(1)
WS2002x[1] = wsx.count(2)
WS2002x[2] = wsx.count(3)
WS2002x[3] = wsx.count(4)
WS2002x[4] = wsx.count(5)
WS2002x[5] = wsx.count(6)
WS2002x[6] = wsx.count(7)
WS2002x[7] = wsx.count(8)
WS2002x[8] = wsx.count(9)
WS2002x[9] = wsx.count(10)
WS2002x[10] = wsx.count(11)

print(WS2002, WS2002x)

[ 1980.  6364.  5523.   423.  3718.  1479. 20480.  4888.  3239.  1075.
  1408.] [ 1566.  7709.  5409.  2891.  4915.  2148. 19885.  4475.  2316.  1237.
  1844.]


## 2003

In [136]:
#2003
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [137]:
fn = r"WS\2003.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [138]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2003\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2003\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX

In [139]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2003\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2003\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [140]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2003\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2003\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [141]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [142]:
WS2003 = np.zeros(11)  # Count occurences of each WS for with DI
WS2003[0] = ws.count(1)
WS2003[1] = ws.count(2)
WS2003[2] = ws.count(3)
WS2003[3] = ws.count(4)
WS2003[4] = ws.count(5)
WS2003[5] = ws.count(6)
WS2003[6] = ws.count(7)
WS2003[7] = ws.count(8)
WS2003[8] = ws.count(9)
WS2003[9] = ws.count(10)
WS2003[10] = ws.count(11)

WS2003x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS2003x[0] = wsx.count(1)
WS2003x[1] = wsx.count(2)
WS2003x[2] = wsx.count(3)
WS2003x[3] = wsx.count(4)
WS2003x[4] = wsx.count(5)
WS2003x[5] = wsx.count(6)
WS2003x[6] = wsx.count(7)
WS2003x[7] = wsx.count(8)
WS2003x[8] = wsx.count(9)
WS2003x[9] = wsx.count(10)
WS2003x[10] = wsx.count(11)

print(WS2003, WS2003x)

[ 3862.  7455.  2682.   354.  5411.   935. 14531.  6715.  3287.  1764.
   622.] [ 4873.  9397.  4686.  2551.  4217.  1319. 18386.  4648.  2136.   976.
   914.]


## 2004

In [143]:
#2004
a = -1  # counter for Jan
b = 123  # counter for Feb
c = 247  # counter for Dec

WS_all = np.zeros((373,181,360))
WSX_all = np.zeros((373,181,360))

In [144]:
fn = r"WS\2004.nc"
ds = nc.Dataset(fn)

lon = np.linspace(-180, 180, 360)
lat = np.linspace(-90, 90, 180)
ws = ds.variables['ws'][:]

jan = ds.variables['january'][:]
jan6hr = jan[0::2]
jan6hr = np.ma.filled(jan6hr, fill_value=0)
jantranslated = np.roll(jan6hr, 180, axis=2)
jansize = np.size(jan6hr, 0)

feb = ds.variables['february'][:]
feb6hr = feb[0::2]
feb6hr = np.ma.filled(feb6hr, fill_value=0)
febtranslated = np.roll(feb6hr, 180, axis=2)
febsize = np.size(feb6hr, 0)

dec = ds.variables['december'][:]
dec6hr = dec[0::2]
dec6hr = np.ma.filled(dec6hr, fill_value=0)
dectranslated = np.roll(dec6hr, 180, axis=2)
decsize = np.size(dec6hr, 0)

In [145]:
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2004\JAN"):  # Run through the files in each folder
    a += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2004\JAN", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = jantranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = jantranslated[a,j,i]
                    
        WS_all[a,:,:] = WS
        WSX_all[a,:,:] = WSX


In [146]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2004\FEB"):  # Run through the files in each folder
    a += 1
    b += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2004\FEB", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = febtranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = febtranslated[a,j,i]

        WS_all[b,:,:] = WS
        WSX_all[b,:,:] = WSX

In [147]:
a = -1
for filename in os.listdir("DI_FRONT\DJF1983-2004\DJF_2004\DEC"):  # Run through the files in each folder
    a += 1
    c += 1
    if filename.endswith(".nc"):            
        fn1 = os.path.join("DI_FRONT\DJF1983-2004\DJF_2004\DEC", filename)
        ds1 = nc.Dataset(fn1)

        lat1 = ds1.variables['latitude'][:]  # Define all relevant variables
        lon1 = ds1.variables['longitude'][:]
        ct = ds1.variables['coldtrail'][:]
        dic = ds1.variables['di_coldtrail'][:]
        cdi = ds1.variables['coldtrail_di'][:]
        
        WS = np.zeros((cdi.shape[0], cdi.shape[1]))  # shape must match cdi
        WSX = np.zeros((cdi.shape[0], cdi.shape[1]))

        for i in range(minlat, maxlat):
            for j in range(minlong,maxlong):  # only choose points in desired latitude and longitude range
                if cdi[i,j] != 0:
                    WS[i,j] = dectranslated[a,j,i]                    
                elif ct[i,j] != 0:
                    WSX[i,j] = dectranslated[a,j,i]
                    
        WS_all[c,:,:] = WS
        WSX_all[c,:,:] = WSX

In [148]:
ws = WS_all[np.nonzero(WS_all)]  # Take only nonzero elements
ws = ws.tolist()
wsx = WSX_all[np.nonzero(WSX_all)]  # Same for w/o DIs
wsx = wsx.tolist()

In [149]:
WS2004 = np.zeros(11)  # Count occurences of each WS for with DI
WS2004[0] = ws.count(1)
WS2004[1] = ws.count(2)
WS2004[2] = ws.count(3)
WS2004[3] = ws.count(4)
WS2004[4] = ws.count(5)
WS2004[5] = ws.count(6)
WS2004[6] = ws.count(7)
WS2004[7] = ws.count(8)
WS2004[8] = ws.count(9)
WS2004[9] = ws.count(10)
WS2004[10] = ws.count(11)

WS2004x = np.zeros(11)  # Count occurences of each WS for w/o DI
WS2004x[0] = wsx.count(1)
WS2004x[1] = wsx.count(2)
WS2004x[2] = wsx.count(3)
WS2004x[3] = wsx.count(4)
WS2004x[4] = wsx.count(5)
WS2004x[5] = wsx.count(6)
WS2004x[6] = wsx.count(7)
WS2004x[7] = wsx.count(8)
WS2004x[8] = wsx.count(9)
WS2004x[9] = wsx.count(10)
WS2004x[10] = wsx.count(11)

print(WS2004, WS2004x)

[ 4252.  9333.  3487.   263.  4548.  1069. 15778.  6171.  2891.  1711.
   894.] [ 4686. 11465.  5412.  2699.  4729.  1768. 18872.  4240.  1787.  1066.
  1022.]


In [150]:
pd.DataFrame(data=[WS1984, WS1984x, WS1985, WS1985x, WS1986, WS1986x, WS1987, WS1987x, WS1988, WS1988x, WS1989, WS1989x, WS1990,\
    WS1990x, WS1991, WS1991x, WS1992, WS1992x, WS1993, WS1993x, WS1995, WS1995x, WS1996, WS1996x, WS1997, WS1997x, WS1998, WS1998x,\
            WS1999, WS1999x, WS2000, WS2000x, WS2001, WS2001x, WS2002, WS2002x, WS2003, WS2003x, WS2004, WS2004x],\
          index = ["1984 With DI"," 1984 W/O DIs","1985 With DI"," 1985 W/O DIs","1986 With DI"," 1986 W/O DIs","1987 With DI"," 1987 W/O DIs","1988 With DI",\
              " 1988 W/O DIs","1989 With DI"," 1989 W/O DIs","1990 With DI"," 1990 W/O DIs","1991 With DI"," 1991 W/O DIs","1992 With DI"," 1992 W/O DIs",\
                  "1993 With DI"," 1993 W/O DIs","1995 With DI"," 1995 W/O DIs","1996 With DI"," 1996 W/O DIs","1997 With DI"," 1997 W/O DIs","1998 With DI",\
                      " 1998 W/O DIs","1999 With DI"," 1999 W/O DIs","2000 With DI"," 2000 W/O DIs","2001 With DI"," 2001 W/O DIs","2002 With DI"," 2002 W/O DIs",\
                          "2003 With DI"," 2003 W/O DIs","2004 With DI","2004 W/O DIs",], columns = ['1', '2', '3', '4', '5','6','7','8','9','10','11']).to_csv('DJF.csv')
